# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# Подргужаем библиотеки
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
# Загружаем датасет и ознакомимся с ним
df = pd.read_csv('/datasets/insurance.csv')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
print('Дубликатов до удаления:', df.duplicated().sum())
df = df.drop_duplicates()
print('Дубликатов после удаления:', df.duplicated().sum())

Дубликатов до удаления: 153
Дубликатов после удаления: 0


## Алгоритм преобразования

**Алгоритм**

1. Сгенерировать квадратную обратную матрицу 4х4
2. Умноженить исходные признаки на обратную матрицу, с целью зашифровать данные.
3. Посчитать качество модели на полученных данных.
4. Сравнить значения метрики с исходными данными.

**Обоснование**

$$ a1 = XPP^{-1}w $$
$$ a1 = X1w1 $$
$$ a1 = Xw $$

## Проверка алгоритма

In [4]:
# Создадим наши исходные признаки
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [5]:
# Сгенерируем квадратную матрицу 4х4
matrix_generated = np.random.normal(size=(4, 4))
matrix_generated

array([[ 0.27259277,  0.14519579,  1.26071006, -0.42022226],
       [-0.11853097,  0.44923043,  0.78157154,  1.38672744],
       [ 1.41126441, -1.58360202, -1.89176668,  0.17338857],
       [-0.10217673,  0.11533071, -1.15999319, -0.26272187]])

In [6]:
# Обратим матрицу
matrix_invert = np.linalg.inv(matrix_generated)
matrix_invert

array([[ 1.89713125,  0.83858689,  0.53910876,  1.74766553],
       [ 1.55878759,  0.93364563, -0.04871788,  2.40264592],
       [ 0.07510218, -0.10557221, -0.07604914, -0.72755852],
       [-0.38513967,  0.54984758,  0.1047246 , -0.21889675]])

In [7]:
# Преобразуем признаки в матирцу
matrix = features.values
# Умножим полученную матрицу на нашу обратную
matrix_new = matrix@matrix_invert
features_incode = pd.DataFrame (matrix_new, columns = features.columns)
features_incode.head(5)

,Пол,Возраст,Зарплата,Члены семьи
0,3790.490656,-5196.713515,-3773.390728,-35986.865357
1,2925.202117,-3968.246283,-2892.003452,-27536.920957
2,1622.350724,-2189.940604,-1598.444667,-15209.052195
3,3163.725372,-4381.654739,-3172.062583,-30289.172527
4,2005.710211,-2728.453913,-1985.707432,-18920.255629


In [8]:
# Создаем класс линейной регрессии
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [9]:
# Обучаем модель на исходных данных
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
original = r2_score(target, predictions)
original

0.4302010044852068

In [10]:
# Также на преобразованных данных
model = LinearRegression()
model.fit(features_incode, target)
predictions = model.predict(features_incode)
incode = r2_score(target, predictions)
incode

0.4302010044852056

Разница в результах метрик R2 бесконечно стремиться к нулю. Могу предположить это связанно с тем, что юпитер округлил значения во время преобразования данных.

**Вывод:** После ознакомления с датасетом были удалены дубликаты. После работы формулой было выявленно, что качество линейной регрессии не меняется, но меняются веса для новых признаков (w1). Я сгенерировал квадратную обратную матрицу 4х4. Умноженил исходные признаки на обратную матрицу, с целью зашифровать данные. Расчитал качество модели на полученных данных и сравнил значения метрики с исходными данными. Разницы в значениях не обнаружил. Таким образом можно сделать вывод, что качество модели не изменилось, оно такое же как и на исходных данных. Значит данный метод можно использовать с целью зашифровать данные.